In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from rubiks_network_experimental import student_network
from utils import dotdict
from keras import regularizers
from custom_layers.unit_regularizer import UnitRegularizer

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "residual_layers" : 2
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : UnitRegularizer(5e-4,5e-4),
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params,
    "state_size":task.input_size,
    "action_codes": task.n_actions
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load("models/combined/trained/trained")
student_template = student(task, 50,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=5,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2)))

In [2]:
c.max_steps=10
c.buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2))
t.step_dist = lambda n : 5*np.ones(n,dtype=int)

In [3]:

for i in range(50):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 100 out of 100 remain open.
Before step 2, 100 out of 100 remain open.
Before step 3, 99 out of 100 remain open.
Before step 4, 99 out of 100 remain open.
Before step 5, 98 out of 100 remain open.
Before step 6, 98 out of 100 remain open.
Before step 7, 98 out of 100 remain open.
Before step 8, 98 out of 100 remain open.


KeyboardInterrupt: 

In [2]:
n=8192
rng = np.random.default_rng()
states = np.arange(task.input_size)[None,:].repeat(n,axis=0)

for k in range(80):
    print(f"Round {k}.")
    #c.run_training_batch(n_problems=100,epochs_per_episode=1)

    for i in range(50):
        _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

    actions = rng.choice(a=task.n_actions,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors.reshape(n,54,6),epochs = 1)

    #if k % 10 == 9:
    #  my_student_network.save("models/combined/trained")

Round 0.


TypeError: in user code:

    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\leove\OneDrive\Documents\Selberprogrammiertes\Taskmaster\rubiks_network_experimental.py", line 34, in crossentropy_loss  *
        return -tf.reduce_mean(tf.square(tf.reduce_sum(y_true*tf.math.log(y_pred+1e-5),axis=-1),axis=-1))

    TypeError: square() got an unexpected keyword argument 'axis'


In [ ]:
from rubiks import print_coloring
states =  np.arange(54,dtype=int)[None,:]
for i in range(50):
    _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=1))
actions = np.array([9],dtype=int)
prediction = my_student_network.predict_state(task.make_input(states),(np.arange(task.n_actions) == actions[...,None]).astype(float))
print_coloring(prediction.reshape(54,6).argmax(axis=-1),color_array=True)
print_coloring(rubiks.task_action(states,actions)[1][0])

In [3]:
n=8192
rng = np.random.default_rng()
states = np.arange(task.input_size)[None,:].repeat(n,axis=0)

for i in range(50):
    _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

actions = rng.choice(a=task.n_actions,size=n)
_, next_states = rubiks.task_action(states,actions)

actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
colors = rubiks.make_neural_input(states)
next_colors = rubiks.make_neural_input(next_states)
results = my_student_network.predict_state(colors,actions).argmax(axis=-1)

same = np.all(next_colors.reshape(n,54,6).argmax(axis=-1) == results,axis=-1)
same.sum()/same.size

256/256 [==============================] - 2s 8ms/step


0.1656494140625

In [7]:
model=my_student_network.state_network
w=model.layers[3].weights[0].numpy()
v=w.copy()
#np.count_nonzero(w<1e-2)/w.size
v[np.abs(w)<0.05]=0
np.abs(v).mean(), np.abs(w).mean()
#print(w.max(axis=1))

(0.001356167, 0.0035329817)

In [59]:
model.trainable_weights

[<tf.Variable 'dense_4/kernel:0' shape=(342, 1792) dtype=float32, numpy=
 array([[-2.57139545e-05,  8.21412250e-05,  2.37136250e-04, ...,
          1.61647375e-04, -2.57835584e-03, -2.80978184e-05],
        [-7.47510494e-05,  1.55858961e-05,  9.51578768e-05, ...,
         -1.04722660e-03,  6.53745781e-04,  2.53204897e-04],
        [-9.82366691e-06, -6.71738380e-05, -9.20492748e-05, ...,
         -1.74514332e-03, -6.69939793e-04,  4.70840168e-04],
        ...,
        [ 2.10733720e-04,  5.91348362e-05, -9.66695879e-05, ...,
         -6.17034530e-05, -1.23539765e-03, -4.61782765e-05],
        [ 2.70831952e-04,  3.97722761e-05, -2.37967557e-04, ...,
          1.17846322e-03, -2.27686280e-04,  2.58572283e-04],
        [ 1.10749395e-04,  1.16978961e-04, -2.14756859e-04, ...,
          6.37816702e-05, -7.82860094e-04, -3.19042410e-05]], dtype=float32)>,
 <tf.Variable 'dense_4/bias:0' shape=(1792,) dtype=float32, numpy=
 array([ 0.00803837, -0.0634234 , -0.2345813 , ...,  0.42080772,
        